In [1]:
from langchain_community.tools import WikipediaQueryRun

In [2]:
from langchain_community.utilities import WikipediaAPIWrapper

In [10]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
wiki_tool=WikipediaQueryRun(api_wrapper=api_wrapper)
wiki_tool

WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/Users/venkatadaggupati/Documents/github/GenAI/LangChain/env/lib/python3.10/site-packages/wikipedia/__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200))

In [4]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()
chunks= RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
embeddings=OpenAIEmbeddings()
vectordb = FAISS.from_documents(chunks,embeddings)

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [5]:
#Saving Embeddings into a pkl file to use it later
chunks = [str(chunk) for chunk in chunks]
chunk_embeddings1 = embeddings.embed_documents(chunks)
data_with_embeddings1 = list(zip(chunks, chunk_embeddings1))
import pickle
with open('Langsmith_data_with_embeddings.pkl', 'wb') as f:
    pickle.dump(data_with_embeddings1, f)

In [6]:
retriever = vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7ff3b9fc04f0>)

In [11]:
#create a tool for retriever
from langchain.tools.retriever import create_retriever_tool
web_retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for any information about langsmith. For any enquiries about Langsmith, you should use this tool"
)


In [8]:
web_retriever_tool.name

'langsmith_search'

In [9]:
#Tool for Arxiv
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper = ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=200)
arxiv_tool= ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv_tool.name

'arxiv'

In [12]:
tools=[arxiv_tool,web_retriever_tool,wiki_tool]

In [13]:
from dotenv import load_dotenv
load_dotenv()
import os
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-3.5-turbo-0125",temperature=0) 

In [14]:
from langchain import hub
# Getting prompts from hub - we can modify it as per our usecase
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [15]:
#Agent
from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(llm,tools,prompt)

In [16]:
#Agent Executor
from langchain.agents import AgentExecutor
agent_executor= AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-functions-agent', 'lc_hub_commit_hash': 'a165

In [17]:
agent_executor.invoke({"input":"Tell me about Langsmith"})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'Langsmith'}`


Get started with LangSmith | 🦜️🛠️ LangSmith

Skip to main contentLangSmith API DocsSearchGo to AppQuick startTutorialsHow-to guidesConceptsReferencePricingSelf-hostingQuick startOn this pageGet started with LangSmithLangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!1. Install LangSmith​PythonTypeScriptpip install -U langsmithyarn add langchain langsmith2. Create an API key​To create an API key head to the Settings page. Then click Create API Key.3. Set up your environment​Shellexport LANGCHAIN_TRACING_V2=trueexport LANGCHAIN_API_KEY=<your-api-key># The below examples use the OpenAI API, though it's not necessary in generalexport OPENAI_API_KEY=<your-openai-api-key>4. Log your first trace​We prov

{'input': 'Tell me about Langsmith',
 'output': 'LangSmith is a platform for building production-grade LLM (Large Language Model) applications. It allows you to closely monitor and evaluate your application, enabling you to ship quickly and with confidence. LangSmith works independently and does not require the use of LangChain. Here are the steps to get started with LangSmith:\n\n1. Install LangSmith:\n   - Python: `pip install -U langsmith`\n   - TypeScript: `yarn add langchain langsmith`\n\n2. Create an API key:\n   - Head to the Settings page and click on Create API Key.\n\n3. Set up your environment:\n   - Set environment variables:\n     ```\n     export LANGCHAIN_TRACING_V2=true\n     export LANGCHAIN_API_KEY=<your-api-key>\n     export OPENAI_API_KEY=<your-openai-api-key> (optional)\n     ```\n\n4. Log your first trace:\n   - LangSmith provides multiple ways to log traces. You can use the provided code snippets to log your first trace.\n\nFor more detailed information and tutor

In [18]:
agent_executor.invoke({"input":"Tell me the future of GenAI"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'GenAI'}`


Page: Generative artificial intelligence
Summary: Generative artificial intelligence (generative AI, GenAI, or GAI) is artificial intelligence capable of generating text, images, videos, or other dataGenerative artificial intelligence (GenAI) refers to artificial intelligence that can generate text, images, videos, or other data. As a field, GenAI is continuously evolving and has the potential to revolutionize various industries by enabling machines to create content autonomously. The future of GenAI holds promise in areas such as creative content generation, personalized recommendations, and automation of tasks that require creativity and innovation. It is expected to play a significant role in shaping the future of technology and how we interact with AI-driven systems.

> Finished chain.


{'input': 'Tell me the future of GenAI',
 'output': 'Generative artificial intelligence (GenAI) refers to artificial intelligence that can generate text, images, videos, or other data. As a field, GenAI is continuously evolving and has the potential to revolutionize various industries by enabling machines to create content autonomously. The future of GenAI holds promise in areas such as creative content generation, personalized recommendations, and automation of tasks that require creativity and innovation. It is expected to play a significant role in shaping the future of technology and how we interact with AI-driven systems.'}

In [20]:
agent_executor.invoke({"input":"How to implement this  research paper 2406.10196"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '2406.10196'}`


Published: 2024-06-14
Title: TRIP-PAL: Travel Planning with Guarantees by Combining Large Language Models and Automated Planners
Authors: Tomas de la Rosa, Sriram Gopalakrishnan, Alberto Pozanco, ZhenThe research paper with the identifier 2406.10196 is titled "TRIP-PAL: Travel Planning with Guarantees by Combining Large Language Models and Automated Planners." It is authored by Tomas de la Rosa, Sriram Gopalakrishnan, Alberto Pozanco, and Zhen. The paper focuses on implementing a system called TRIP-PAL that combines large language models and automated planners for travel planning with guarantees.

> Finished chain.


{'input': 'How to implement this  research paper 2406.10196',
 'output': 'The research paper with the identifier 2406.10196 is titled "TRIP-PAL: Travel Planning with Guarantees by Combining Large Language Models and Automated Planners." It is authored by Tomas de la Rosa, Sriram Gopalakrishnan, Alberto Pozanco, and Zhen. The paper focuses on implementing a system called TRIP-PAL that combines large language models and automated planners for travel planning with guarantees.'}